In [1]:
import cv2
import numpy as np
from keras.models import model_from_json
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import threading

# Load the model architecture from the JSON file
json_file_path = r"C:\Users\delld\emotion_model.json"
weights_file_path = r"C:\Users\delld\emotion_model.weights.h5"

with open(json_file_path, 'r') as json_file:
    loaded_model_json = json_file.read()

emotion_model = model_from_json(loaded_model_json)

# Check if the weights file exists and load weights
if os.path.exists(weights_file_path):
    emotion_model.load_weights(weights_file_path)
    print("Model weights loaded successfully.")
else:
    print(f"File not found: {weights_file_path}")

# Compile the model
emotion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Emotion labels
emotion_dict = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# Load the face detection cascade
cascade_path = r"C:\Users\delld\haarcascade_frontalface_default.xml"
face_detector = cv2.CascadeClassifier(cascade_path)

# Check if the cascade was loaded correctly
if face_detector.empty():
    print(f"Error: Could not load cascade file at {cascade_path}")
    exit()


Model weights loaded successfully.


In [2]:
def detect_emotion(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        return frame, faces

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    return frame, faces


In [3]:
def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        image_path.set(file_path)
        img = cv2.imread(file_path)
        if img is not None:
            detected_image, faces = detect_emotion(img)
            display_image(detected_image)
            if len(faces) == 0:
                messagebox.showinfo("Result", "No faces detected.")
            else:
                messagebox.showinfo("Result", f"Faces detected: {len(faces)}\nEmotions predicted.")
        else:
            messagebox.showerror("Error", "Failed to load image.")

def start_webcam():
    global webcam_running
    webcam_running = True
    webcam_thread = threading.Thread(target=webcam_loop)
    webcam_thread.start()

def stop_webcam():
    global webcam_running
    webcam_running = False

def webcam_loop():
    global webcam_running
    cap = cv2.VideoCapture(0)
    while webcam_running:
        ret, frame = cap.read()
        if not ret:
            break
        frame, _ = detect_emotion(frame)
        display_image(frame)
    cap.release()

def display_image(cv_img):
    cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv_img)
    imgtk = ImageTk.PhotoImage(image=img)
    panel.imgtk = imgtk
    panel.config(image=imgtk)


In [4]:
# Initialize the main window
root = tk.Tk()
root.title("Emotion Detection")
root.geometry("800x600")
root.configure(bg="#f0f0f0")

# Define the image path variable
image_path = tk.StringVar()

# Create a frame for the controls
frame_controls = tk.Frame(root, bg="#f0f0f0")
frame_controls.pack(pady=10)

# Create buttons for image and webcam
btn_select_image = tk.Button(frame_controls, text="Select Image", command=select_image, bg="#4CAF50", fg="white", font=("Arial", 12))
btn_select_image.grid(row=0, column=0, padx=10, pady=10)

btn_start_webcam = tk.Button(frame_controls, text="Start Webcam", command=start_webcam, bg="#2196F3", fg="white", font=("Arial", 12))
btn_start_webcam.grid(row=0, column=1, padx=10, pady=10)

btn_stop_webcam = tk.Button(frame_controls, text="Stop Webcam", command=stop_webcam, bg="#F44336", fg="white", font=("Arial", 12))
btn_stop_webcam.grid(row=0, column=2, padx=10, pady=10)

# Create a label to display the image
panel = tk.Label(root)
panel.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

# Add a status bar
status_bar = tk.Label(root, text="Ready", anchor="w", relief=tk.SUNKEN, background="#d9d9d9", font=("Arial", 10))
status_bar.pack(side=tk.BOTTOM, fill=tk.X)

# Initialize webcam running status
webcam_running = False

# Start the GUI event loop
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━